In [20]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 16.1 MB/s 
     |████████████████████████████████| 84 kB 4.0 MB/s 
     |████████████████████████████████| 140 kB 73.3 MB/s 
     |████████████████████████████████| 84 kB 5.0 MB/s 
     |████████████████████████████████| 2.3 MB 54.8 MB/s 
     |████████████████████████████████| 54 kB 4.5 MB/s 
     |████████████████████████████████| 57 kB 6.3 MB/s 
     |████████████████████████████████| 1.1 MB 37.1 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 212 kB 71.3 MB/s 
     |████████████████████████████████| 272 kB 60.4 MB/s 
     |████████████████████████████████| 144 kB 59.5 MB/s 
     |████████████████████████████████| 271 kB 51.8 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 80 kB

In [5]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline


In [6]:
test_transform = transforms.Compose([
                  transforms.Resize(227),
                  transforms.CenterCrop(227),
                  transforms.ToTensor(),
                  transforms.Normalize([0.485, 0.456, 0.406],
                                       [0.229, 0.224, 2.225])
])

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
testing_root = "/content/drive/MyDrive/Capstone/Dataset_1/test"

In [9]:


test_data = datasets.ImageFolder(testing_root, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

class_names = test_data.classes

print(class_names)

print(f'Testing images available:  {len(test_data)}')

['N95', 'N95Valve', 'NoMask', 'cloth', 'surgical']
Testing images available:  500


In [11]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.conv3 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(26*26*64, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 5)
        self.norm1 = nn.BatchNorm2d(16)
        self.norm2 = nn.BatchNorm2d(32)
        self.norm3 = nn.BatchNorm2d(64)
        self.dropOut = nn.Dropout(p=0.2)

    def forward(self, X):
        # print(X.shape)
        X = F.leaky_relu(self.conv1(X)) 
        # print(X.shape)
        X = F.max_pool2d(X, 2, 2)
        X = self.norm1(X)  
        # print(X.shape)

        X = F.leaky_relu(self.conv2(X))
        # print(X.shape)
        X = F.max_pool2d(X, 2, 2)
        X = self.norm2(X)
        # print(X.shape)

        X = X.view(-1, 26*26*64)
        # print(X.shape)
        X = F.leaky_relu(self.fc1(X))
        # print(X.shape)
        X = self.dropOut(X)
        # print(X.shape)
        X = F.leaky_relu(self.fc2(X))
        # print(X.shape)
        X = self.dropOut(X)
        # print(X.shape)
        X = F.leaky_relu(self.fc3(X))
        # print(X.shape)
        X = self.dropOut(X)
        # print(X.shape)
        X = self.fc4(X)
        # print(X.shape,"done")
        return F.log_softmax(X, dim=1)

In [12]:
model = ConvolutionalNetwork()
model.load_state_dict(torch.load('/content/drive/MyDrive/Capstone/FinalmaskModel.pt')) 

<All keys matched successfully>

In [13]:
!pip install replicate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for replicate: filename=replicate-0.0.1a15-py3-none-any.whl size=14075 sha256=3f1435214b40b2beb1dffd211f6aaad41066a89bf098a9bf2303d3d8a814c3b0
  Stored in directory: /root/.cache/pip/wheels/a6/40/af/6d663eaa2f115c9c772d82df5265b0a451154f7753cf4a5c80
Successfully built replicate


In [16]:
#import replicate
#model = replicate.models.get("/content/drive/MyDrive/Capstone/FinalmaskModel.pt")

In [17]:
model.eval()

ConvolutionalNetwork(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=43264, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=5, bias=True)
  (norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropOut): Dropout(p=0.2, inplace=False)
)

In [21]:
import gradio as gr

In [22]:
def predict_image(img):
  img_4d=img.reshape(-1,48,48,3)
  prediction=model(img_4d)[0]
  return {class_names[i]: float(prediction[i]) for i in range(5)}

In [ ]:
image = gr.inputs.Image(shape=(48,48))
label = gr.outputs.Label(num_top_classes=3)

gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(debug='True')


/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  "Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
 

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://58383.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/gradio/routes.py", line 256, in run_predict
    fn_index, raw_input, username, session_state
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 599, in process_api
    predictions, duration = await self.call_function(fn_index, processed_input)
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 515, in call_function
    block_fn.fn, *processed_input, limiter=self.limiter
  File "/usr/local/lib/python3.7/dist-packages/anyio/to_thread.py", line 32, in run_sync
    func, *args, cancellable=cancellable, limiter=limiter
  File "/usr/local/lib/python3.7/dist-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.7/dist-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/usr/local/lib/python3.7/dist-packages/gradio/interface.py", line 487, in <lam